In [132]:
import os
import subprocess
import matplotlib.pyplot as plt
import numpy as np
import sys
import threading
import csv

In [22]:
tsp_dir = os.getcwd() + os.sep + 'TSPLIB' + os.sep

In [23]:
tsp_files_list = os.listdir(tsp_dir)

In [24]:
subprocess.run('make christofides', shell=True, capture_output=True)

CompletedProcess(args='make christofides', returncode=0, stdout=b"make: 'christofides' is up to date.\n", stderr=b'')

In [25]:
out = subprocess.run(['./christofides -f TSPLIB/gr17.tsp'], shell=True, capture_output=True)
output = out.stdout.decode('utf-8')
print(output)

OptimalSoln:	2085.000000
Simple:		1.053717	2197
Tri-Opt:	1.031655	2151
Tri-Comp:	1.031655	2151
Comp-Heur:	1.074820	2241



In [26]:
def parseOutputString(outputStr):
    lines = outputStr.split('\n')
    resultDict = {}
    for i in range(len(lines)-1):
        line = lines[i]
        s = line.split()
        if(i == 0):
            resultDict[i] = [s[1]]
        else:
            resultDict[i] = s[1:]
    return resultDict

In [27]:
parseOutputString(output)

{0: ['2085.000000'],
 1: ['1.053717', '2197'],
 2: ['1.031655', '2151'],
 3: ['1.031655', '2151'],
 4: ['1.074820', '2241']}

In [69]:
def getSupportedFilesList():
    tsp_files_list = os.listdir(tsp_dir)
    supportedFiles = []
    keyWord = 'EDGE_WEIGHT_SECTION'
    for filename in tsp_files_list:
        if(filename.find('.tsp') == -1):
            continue
        foundKeyWord = False
        file = open(tsp_dir + os.sep + filename, 'r')
        lines = file.readlines()
        for line in lines:
            if(line.find(keyWord) != -1):
                foundKeyWord = True
                break
        if(foundKeyWord == False):
            supportedFiles.append(filename)
    return supportedFiles

In [70]:
supported_tsp_files_list = getSupportedFilesList()

In [30]:
def launchChristofides(filename = 'gr48.tsp', perturbation_percentage = 0.0):
    filename = filename
    if(filename not in supported_tsp_files_list):
        print('WARNING: You are running the algorithm on an unsupported file, Perturbation does not work.')
    filename = 'TSPLIB' + os.sep + filename
    command = './christofides' + ' -f ' + filename + ' -p ' + str(perturbation_percentage)
#     print(command)
    out = subprocess.run([command], shell=True, capture_output=True)
    output = out.stdout.decode('utf-8')
    resultDict = parseOutputString(output)
#     print(resultDict)
    return resultDict

In [31]:
launchChristofides('gr431.tsp', 0.01)

{0: ['171414.000000'],
 1: ['1.125492', '192925'],
 2: ['1.070642', '183523'],
 3: ['1.050603', '180088'],
 4: ['1.079883', '185107']}

In [32]:
MAX_THREADS = 1

threads = []
for i in range(MAX_THREADS):
    t = threading.Thread(target = launchChristofides, args = ('gr431', 0.01))
    threads.append(t)
    t.start()
for t in threads:
    t.join()

In [33]:
def launchChristofidesThreadSafe(resultDictList, listLock, filename = 'gr48', perturbation_percentage = 0.0):
    dictToAdd = launchChristofides(filename, perturbation_percentage)
    listLock.acquire()
    resultDictList.append(dictToAdd)
    listLock.release()
    return    

In [41]:
resultList = []
listLock = threading.Lock()
threads = []
for i in range(10):
    t = threading.Thread(target = launchChristofidesThreadSafe, args = (resultList, listLock, 'gr431.tsp', 0.01))
    threads.append(t)
    t.start()
for t in threads:
    t.join()

print(len(resultList))
print(resultList)

10
[{0: ['171414.000000'], 1: ['1.126192', '193045'], 2: ['1.070228', '183452'], 3: ['1.050258', '180029'], 4: ['1.081569', '185396']}, {0: ['171414.000000'], 1: ['1.125392', '192908'], 2: ['1.070694', '183532'], 3: ['1.050918', '180142'], 4: ['1.086072', '186168']}, {0: ['171414.000000'], 1: ['1.124231', '192709'], 2: ['1.068267', '183116'], 3: ['1.050212', '180021'], 4: ['1.088808', '186637']}, {0: ['171414.000000'], 1: ['1.124762', '192800'], 2: ['1.070134', '183436'], 3: ['1.050748', '180113'], 4: ['1.093866', '187504']}, {0: ['171414.000000'], 1: ['1.118088', '191656'], 2: ['1.064522', '182474'], 3: ['1.051169', '180185'], 4: ['1.080040', '185134']}, {0: ['171414.000000'], 1: ['1.126098', '193029'], 2: ['1.069936', '183402'], 3: ['1.049943', '179975'], 4: ['1.082590', '185571']}, {0: ['171414.000000'], 1: ['1.118835', '191784'], 2: ['1.064785', '182519'], 3: ['1.051664', '180270'], 4: ['1.085827', '186126']}, {0: ['171414.000000'], 1: ['1.124850', '192815'], 2: ['1.068758', '18320

In [35]:
def runExpt(filename, perturbation_percentage, num_threads = 15):
    resultList = []
    listLock = threading.Lock()
    threads = []
    for i in range(num_threads):
        t = threading.Thread(target = launchChristofidesThreadSafe, args = (resultList, listLock, filename, perturbation_percentage))
        threads.append(t)
        t.start()
    for t in threads:
        t.join()
    return resultList

In [36]:
ans = runExpt('gr431.tsp', 0.1, 1)
print(ans)

[{0: ['171414.000000'], 1: ['1.110779', '190403'], 2: ['1.062830', '182184'], 3: ['1.042750', '178742'], 4: ['1.113328', '190840']}]


In [107]:
def getAvgList(inpDict=[]):
    l = [0.0 for i in range(5)]
    n = len(inpDict)
    for d in inpDict:
        try:
            l[0] += float(d[0][0])
        except:
#             print(d)
            n -= 1
#             print('f', n)
            continue
        for i in range(4):
            l[i+1] += float(d[i+1][1])
    if(n == 0):
        print('fhere', end ='\t')
        return l
    for i in range(5):
        l[i] = l[i]/n
    return l

In [38]:
getAvgList(ans)

[171414.0, 190403.0, 182184.0, 178742.0, 190840.0]

In [39]:
NUM_OBSERVATIONS = 100
ppList = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3]
data = {}
# for i in range(len(supported_tsp_files_list)):
for i in range(1):
    filename = supported_tsp_files_list[i]
    for j in range(len(ppList)):
#     for j in range(1):
        pp = ppList[j]
        output = runExpt(filename, pp, NUM_OBSERVATIONS)
#         print(output)
        l = getAvgList(output)
        data[(filename, pp)] = []
        for k in range(5):
            data[(filename, pp)].append(l[k])
print(data)

{('a280.tsp', 0.0): [2579.0, 2909.0, 2711.0, 2617.0, 2815.0], ('a280.tsp', 0.01): [2579.0, 2963.32, 2723.46, 2635.8, 2778.36], ('a280.tsp', 0.02): [2579.0, 2967.14, 2727.36, 2636.86, 2780.47], ('a280.tsp', 0.05): [2579.0, 2970.73, 2730.62, 2642.2, 2767.53], ('a280.tsp', 0.1): [2579.0, 2965.82, 2737.27, 2649.93, 2787.12], ('a280.tsp', 0.2): [2579.0, 2924.53, 2742.66, 2663.02, 2820.93], ('a280.tsp', 0.3): [2579.0, 2921.23, 2745.9, 2669.18, 2835.6]}


In [75]:
def getNumFromString(s):
    numStr = ''
    for c in list(s):
        if(c.isdigit()):
            numStr += (c)
    return int(numStr)

In [110]:
def getData(MAX_N = 1000, NUM_OBSERVATIONS = 15, NUM_FILES = len(supported_tsp_files_list), ppList = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3]):
    data = {}
    if(NUM_FILES > len(supported_tsp_files_list)):
        print('There are only', len(supported_tsp_files_list), 'files available')
        NUM_FILES = len(supported_tsp_files_list)
    for i in range(NUM_FILES):
        filename = supported_tsp_files_list[i]
        if(getNumFromString(filename) > MAX_N):
            continue
        print(filename, end='\t')
        for j in range(len(ppList)):
            pp = ppList[j]
            output = runExpt(filename, pp, NUM_OBSERVATIONS)
            print(pp, end = '\t')
            l = getAvgList(output)
            data[(filename, pp)] = []
            for k in range(5):
                data[(filename, pp)].append(l[k])
        print(u'\u2713')
    return data

In [156]:
##### Uncomment last to run for different parameters, saved variable data is for MAX_N = 1000, NUM_OBS = 20, NUM_FILES = 100
data = []
%store -r data
# data = getData(MAX_N = 1000, NUM_OBSERVATIONS = 20, NUM_FILES = 100)
print(data)

{('a280.tsp', 0.0): [2579.0, 2909.0, 2711.0, 2617.0, 2815.0], ('a280.tsp', 0.01): [2579.0, 2968.5, 2725.05, 2630.85, 2775.75], ('a280.tsp', 0.02): [2579.0, 2971.85, 2728.95, 2640.95, 2787.05], ('a280.tsp', 0.05): [2579.0, 2966.95, 2732.2, 2641.4, 2765.4], ('a280.tsp', 0.1): [2579.0, 2970.5, 2752.0, 2664.45, 2792.05], ('a280.tsp', 0.2): [2579.0, 2918.45, 2744.2, 2664.2, 2846.05], ('a280.tsp', 0.3): [2579.0, 2909.15, 2743.55, 2665.65, 2831.15], ('ali535.tsp', 0.0): [202339.0, 230463.0, 222667.0, 216089.0, 226728.0], ('ali535.tsp', 0.01): [202339.0, 230467.4, 222144.05, 216067.8, 229508.3], ('ali535.tsp', 0.02): [202339.0, 230548.9, 222313.2, 216370.2, 231072.6], ('ali535.tsp', 0.05): [202339.0, 229938.75, 221530.25, 216417.15, 230770.1], ('ali535.tsp', 0.1): [202339.0, 230158.55, 221752.0, 216915.05, 230038.45], ('ali535.tsp', 0.2): [202339.0, 230917.05263157896, 222445.1052631579, 217523.68421052632, 229478.63157894736], ('ali535.tsp', 0.3): [202339.0, 230843.85, 223041.85, 218073.15, 2

In [115]:
a = runExpt('d493.tsp', 0.01, 15)
getAvgList(a)

[35002.0, 38766.5, 37302.25, 36267.75, 38559.75]

In [117]:
print(data)

{('a280.tsp', 0.0): [2579.0, 2909.0, 2711.0, 2617.0, 2815.0], ('a280.tsp', 0.01): [2579.0, 2968.5, 2725.05, 2630.85, 2775.75], ('a280.tsp', 0.02): [2579.0, 2971.85, 2728.95, 2640.95, 2787.05], ('a280.tsp', 0.05): [2579.0, 2966.95, 2732.2, 2641.4, 2765.4], ('a280.tsp', 0.1): [2579.0, 2970.5, 2752.0, 2664.45, 2792.05], ('a280.tsp', 0.2): [2579.0, 2918.45, 2744.2, 2664.2, 2846.05], ('a280.tsp', 0.3): [2579.0, 2909.15, 2743.55, 2665.65, 2831.15], ('ali535.tsp', 0.0): [202339.0, 230463.0, 222667.0, 216089.0, 226728.0], ('ali535.tsp', 0.01): [202339.0, 230467.4, 222144.05, 216067.8, 229508.3], ('ali535.tsp', 0.02): [202339.0, 230548.9, 222313.2, 216370.2, 231072.6], ('ali535.tsp', 0.05): [202339.0, 229938.75, 221530.25, 216417.15, 230770.1], ('ali535.tsp', 0.1): [202339.0, 230158.55, 221752.0, 216915.05, 230038.45], ('ali535.tsp', 0.2): [202339.0, 230917.05263157896, 222445.1052631579, 217523.68421052632, 229478.63157894736], ('ali535.tsp', 0.3): [202339.0, 230843.85, 223041.85, 218073.15, 2

In [137]:
ppList = [0.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3]
allTables = []
for t in range(5):
    table = []
    l1 = []
    l1.append('FileName\Heur: ' + str(t))
    for pp in ppList:
        l1.append(pp)
    table.append(l1)
    for file in supported_tsp_files_list:
#         if(not data.has_key((file, 0.0))):
        if((file, 0.0) not in data.keys()):
            continue
        l = []
        l.append(file)
        for pp in ppList:
            l.append(data[(file, pp)][t])
        table.append(l)
    allTables.append(table)
print(len(allTables[0]))

63


In [148]:
def writeCsvTableToDisk(filename, table):
    with open(filename+'.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerows(table)
    return

In [149]:
for t in range(len(allTables)):
    writeCsvTableToDisk('Heuristic '+ str(t), allTables[t])

In [150]:
print(allTables[0])

[['FileName\\Heur: 0', 0.0, 0.01, 0.02, 0.05, 0.1, 0.2, 0.3], ['a280.tsp', 2579.0, 2579.0, 2579.0, 2579.0, 2579.0, 2579.0, 2579.0], ['ali535.tsp', 202339.0, 202339.0, 202339.0, 202339.0, 202339.0, 202339.0, 202339.0], ['att48.tsp', 10628.0, 10628.0, 10628.0, 10628.0, 10628.0, 10628.0, 10628.0], ['att532.tsp', 27686.0, 27686.0, 27686.0, 27686.0, 27686.0, 27686.0, 27686.0], ['berlin52.tsp', 7542.0, 7542.0, 7542.0, 7542.0, 7542.0, 7542.0, 7542.0], ['bier127.tsp', 118282.0, 118282.0, 118282.0, 118282.0, 118282.0, 118282.0, 118282.0], ['burma14.tsp', 3323.0, 3323.0, 3323.0, 3323.0, 3323.0, 3323.0, 3323.0], ['ch130.tsp', 6110.0, 6110.0, 6110.0, 6110.0, 6110.0, 6110.0, 6110.0], ['ch150.tsp', 6528.0, 6528.0, 6528.0, 6528.0, 6528.0, 6528.0, 6528.0], ['d198.tsp', 15780.0, 15780.0, 15780.0, 15780.0, 15780.0, 15780.0, 15780.0], ['d493.tsp', 0.0, 35002.0, 35002.0, 35002.0, 35002.0, 35002.0, 35002.0], ['d657.tsp', 48912.0, 48912.0, 48912.0, 48912.0, 48912.0, 48912.0, 48912.0], ['dsj1000.tsp', 186601

In [151]:
%store data

Stored 'data' (dict)
